## Imports

In [2]:
import dostools
import importlib
import numpy as np
import pickle
import torch
import sys
import matplotlib.pyplot as plt
torch.set_default_dtype(torch.float64) 

In [6]:
import dostools.src.datasets.data as data
import dostools.src.utils.utils as utils

n_structures = 1039
np.random.seed(0)
n_train = int(0.8 * n_structures)
train_index = np.arange(n_structures)
np.random.shuffle(train_index)
test_index = train_index[n_train:]
train_index = train_index[:n_train]

with torch.no_grad():
    structures = data.load_structures(":")
    n_structures = len(structures) #total number of structures
    for structure in structures:#implement periodicity
        structure.wrap(eps = 1e-12) 
    n_atoms = np.zeros(n_structures, dtype = int) #stores number of atoms in each structures
    for i in range(n_structures):
        n_atoms[i] = len(structures[i])

    #eigenergies, emin, emax = dostools.src.datasets.data.load_eigenenergies(unpack = True, n_structures = len(structures))
    xdos = torch.tensor(data.load_xdos())
    ldos = torch.tensor(data.load_ldos())
    ldos *= 2

    print ("ldos shape is {}".format(ldos.shape))
    mean_dos_per_atom = ldos[train_index].mean(axis = 0) #only calculated for train set to prevent data leakage
    print ("mean dos shape is {}".format(mean_dos_per_atom.shape))
    
    
    y_pw = ldos - mean_dos_per_atom
    y_lcdf = torch.cumsum(y_pw, dim = 1)
    _, pc_vectors = utils.build_pc(ldos[train_index], mean_dos_per_atom[None,:], n_pc = 10)
    y_pc = utils.build_coeffs(ldos - mean_dos_per_atom[None,:], pc_vectors)
    Silicon = data.load_features()
    kMM = data.load_kMM()

ldos shape is torch.Size([1039, 778])
mean dos shape is torch.Size([778])
Variance covered with 10 PCs is = 0.9871211778950157


## Evaluator

In [7]:
import dostools.src.evaluation.evaluation as evaluation
importlib.reload(evaluation)
import dostools.src.models.training as training
importlib.reload(training)

targets = {
    'pw' : ldos,
    'lcdf' : y_lcdf,
    'pc' : y_pc
}
evaluator = evaluation.Evaluator(targets, xdos, mean_dos_per_atom)

## Analytical Models

In [100]:
import dostools.src.models.models as models
importlib.reload(models)


Ridge_pw_kernel = models.RidgeRegression(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_pw, "pw", xdos)
Ridge_pw_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_pw_kernel.get_DosRMSE(evaluator, train_index, test_index)
Ridge_pw_kernel.get_targetRMSE(evaluator, train_index, test_index)

Ridge_pw_soap = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avesoap", y_pw, "pw", xdos)
Ridge_pw_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index) 
Ridge_pw_soap.get_DosRMSE(evaluator, train_index, test_index) 
Ridge_pw_soap.get_targetRMSE(evaluator, train_index, test_index) 

GPR_pw = models.GPR(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_pw, "pw", xdos, kMM)
GPR_pw.obtain_weights([1.0,0.75,0.5,0.25], train_index)
GPR_pw.get_DosRMSE(evaluator, train_index, test_index)
GPR_pw.get_targetRMSE(evaluator, train_index, test_index)


The performance at reg value 9.999999999999996e-11 is 0.02488911074035507
The performance at reg value 9.999999999999982e-09 is 0.015987433510106452
The performance at reg value 1.0000000000000004e-06 is 0.014727581673490999
The performance at reg value 0.00010000000000000009 is 0.018910912251443146
The performance at reg value 0.010000000000000004 is 0.027055844562503713
The performance at reg value 1.0 is 0.03807414403136081
The performance at reg value 100.00000000000004 is 0.05733310370242155
The log_grid error is:
[0.02488911 0.01598743 0.01472758 0.01891091 0.02705584 0.03807414
 0.0573331 ]
Init value is: 1.0000000000000004e-06
The performance at reg value 1.0000000000000004e-06 is 0.014727581673490999
The performance at reg value 5.011872336272722e-07 is 0.014678866992408106
The performance at reg value 2.511886431509578e-07 is 0.014766975654236724
The performance at reg value 7.07945784384138e-07 is 0.014682803520750489
Optimal regularization value for Ridge, Feature:avekernel

The performance at reg value 9.999999999999996e-11 is 0.02360740219628505
The performance at reg value 9.999999999999982e-09 is 0.041994877468587835
The performance at reg value 1.0000000000000004e-06 is 0.09412310200608882
The performance at reg value 0.00010000000000000009 is 0.0942241889697976
The performance at reg value 0.010000000000000004 is 0.09422419910593646
The performance at reg value 1.0 is 0.0942241991069501
The performance at reg value 100.00000000000004 is 0.0942241991069502
The log_grid error is:
[0.0236074  0.04199488 0.0941231  0.09422419 0.0942242  0.0942242
 0.0942242 ]
Init value is: 9.999999999999996e-11
The performance at reg value [1.e-10] is 0.024605752572204456
The performance at reg value [3.16227766e-11] is 0.02340259997103994
The performance at reg value [1.e-11] is 0.023934284283943095
The performance at reg value [1.77827941e-11] is 0.024046408158937847
The performance at reg value [5.62341325e-11] is 0.023886622098559983
Optimal Regularization value for

In [101]:
Ridge_lcdf_kernel = models.RidgeRegression(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_lcdf, "lcdf", xdos)
Ridge_lcdf_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_lcdf_kernel.get_DosRMSE(evaluator, train_index, test_index)
Ridge_lcdf_kernel.get_targetRMSE(evaluator, train_index, test_index)

Ridge_lcdf_soap = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avesoap", y_lcdf, "lcdf", xdos)
Ridge_lcdf_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index)
Ridge_lcdf_soap.get_DosRMSE(evaluator, train_index, test_index)
Ridge_lcdf_soap.get_targetRMSE(evaluator, train_index, test_index)

GPR_lcdf = models.GPR(Silicon.Features['structure_avekerneldescriptors'], "avekernels", y_lcdf, "lcdf", xdos, kMM)
GPR_lcdf.obtain_weights([1.0,0.75,0.5,0.25], train_index)
GPR_lcdf.get_DosRMSE(evaluator, train_index, test_index)
GPR_lcdf.get_targetRMSE(evaluator, train_index, test_index)

The performance at reg value 9.999999999999996e-11 is 0.30294051089969054
The performance at reg value 9.999999999999982e-09 is 0.24961739104436304
The performance at reg value 1.0000000000000004e-06 is 0.26596574913017423
The performance at reg value 0.00010000000000000009 is 0.3076890689349757
The performance at reg value 0.010000000000000004 is 0.4288955801160411
The performance at reg value 1.0 is 1.0168938487096923
The performance at reg value 100.00000000000004 is 2.0934131122906705
The log_grid error is:
[0.30294051 0.24961739 0.26596575 0.30768907 0.42889558 1.01689385
 2.09341311]
Init value is: 9.999999999999982e-09
The performance at reg value 9.999999999999982e-09 is 0.24961739104436304
The performance at reg value 3.9810717055349665e-09 is 0.2517374519703249
The performance at reg value 2.511886431509575e-08 is 0.2525078253184684
The performance at reg value 6.309573444801934e-09 is 0.2500443950833206
Optimal regularization value for Ridge, Feature:avekernels, target: lcdf

The performance at reg value 9.999999999999996e-11 is 0.3582900919139367
The performance at reg value 9.999999999999982e-09 is 0.37275894760787504
The performance at reg value 1.0000000000000004e-06 is 0.3683979085803729
The performance at reg value 0.00010000000000000009 is 2.628841196282138
The performance at reg value 0.010000000000000004 is 5.318819596087717
The performance at reg value 1.0 is 5.319751341043245
The performance at reg value 100.00000000000004 is 5.319751434245397
The log_grid error is:
[0.35829009 0.37275895 0.36839791 2.6288412  5.3188196  5.31975134
 5.31975143]
Init value is: 9.999999999999996e-11
The performance at reg value [1.e-10] is 0.3617312696251491
The performance at reg value [3.16227766e-11] is 0.37745795705207075
The performance at reg value [3.16227766e-10] is 0.36972685644010284
The performance at reg value [1.77827941e-10] is 0.3745893194886805
The performance at reg value [5.62341325e-11] is 0.36559214078515645
Optimal Regularization value for GPR,

In [ ]:
[369, 341, 745, 521, 278, 5, 193, 37]

## Analytical Evaluation

In [105]:
print ("Ridge_pw_kernel DOS RMSE")
print (Ridge_pw_kernel.DOSrmse)
print ("Ridge_pw_kernel Target RMSE")
print (Ridge_pw_kernel.targetrmse)

Ridge_pw_kernel DOS RMSE
{1.0: array([ 8.94896867, 10.260054  ,  9.21725818,  0.04963029,  0.05958249,
        0.05162265]), 0.75: array([ 9.77312821, 11.12209438, 10.04892532,  0.05420102,  0.06458855,
        0.05628052]), 0.5: array([10.86239665, 11.71137637, 11.03393834,  0.06024202,  0.06801065,
        0.06179724]), 0.25: array([26.12161199, 26.06197913, 26.09796345,  0.14486845,  0.15134789,
        0.14616559])}
Ridge_pw_kernel Target RMSE
{1.0: array([ 8.94896867, 10.260054  ,  9.21725818,  0.04963029,  0.05958249,
        0.05162265]), 0.75: array([ 9.77312821, 11.12209438, 10.04892532,  0.05420102,  0.06458855,
        0.05628052]), 0.5: array([10.86239665, 11.71137637, 11.03393834,  0.06024202,  0.06801065,
        0.06179724]), 0.25: array([26.12161199, 26.06197913, 26.09796345,  0.14486845,  0.15134789,
        0.14616559])}


In [104]:
print ("Ridge_pw_soap DOS RMSE")
print (Ridge_pw_soap.DOSrmse)
print ("Ridge_pw_soap Target RMSE")
print (Ridge_pw_soap.targetrmse)

Ridge_pw_soap DOS RMSE
{1.0: array([10.74889973, 11.6362159 , 10.92844813,  0.05961257,  0.06757417,
        0.06120643]), 0.75: array([11.20180752, 12.09020208, 11.38138437,  0.06212436,  0.07021058,
        0.06374316]), 0.5: array([13.1183806 , 13.29469995, 13.14932045,  0.07275353,  0.07720537,
        0.07364476]), 0.25: array([23.03672075, 22.73477731, 22.9641052 ,  0.12775988,  0.13202607,
        0.12861394])}
Ridge_pw_soap Target RMSE
{1.0: array([10.74889973, 11.6362159 , 10.92844813,  0.05961257,  0.06757417,
        0.06120643]), 0.75: array([11.20180752, 12.09020208, 11.38138437,  0.06212436,  0.07021058,
        0.06374316]), 0.5: array([13.1183806 , 13.29469995, 13.14932045,  0.07275353,  0.07720537,
        0.07364476]), 0.25: array([23.03672075, 22.73477731, 22.9641052 ,  0.12775988,  0.13202607,
        0.12861394])}


In [106]:
print ("GPR_pw DOS RMSE")
print (GPR_pw.DOSrmse)
print ("GPR_pw Target RMSE")
print (GPR_pw.targetrmse)

GPR_pw DOS RMSE
{1.0: array([22.11708717, 21.35773438, 21.94992095,  0.12265967,  0.12402926,
        0.12293385]), 0.75: array([21.5088905 , 20.74365928, 21.20937588,  0.11865888,  0.12046319,
        0.11878631]), 0.5: array([21.06248509, 20.88436152, 21.41177157,  0.11681263,  0.12128028,
        0.11991986]), 0.25: array([23.83315022, 24.49232399, 24.30919548,  0.12613305,  0.14223254,
        0.13614732])}
GPR_pw Target RMSE
{1.0: array([22.11708717, 21.35773438, 21.94992095,  0.12265967,  0.12402926,
        0.12293385]), 0.75: array([21.5088905 , 20.74365928, 21.20937588,  0.11865888,  0.12046319,
        0.11878631]), 0.5: array([21.06248509, 20.88436152, 21.41177157,  0.11681263,  0.12128028,
        0.11991986]), 0.25: array([23.83315022, 24.49232399, 24.30919548,  0.12613305,  0.14223254,
        0.13614732])}


In [109]:
print ("Ridge_lcdf_kernel DOS RMSE")
print (Ridge_lcdf_kernel.DOSrmse)
print ("Ridge_lcdf_kernel Target RMSE")
print (Ridge_lcdf_kernel.targetrmse)

Ridge_lcdf_kernel DOS RMSE
{1.0: array([ 8.21098306, 10.33286287,  8.64789288,  0.04553748,  0.0600053 ,
        0.04843383]), 0.75: array([10.21666806, 11.8486123 , 10.5510134 ,  0.05666086,  0.06880761,
        0.05909255]), 0.5: array([11.17952744, 12.5301001 , 11.45505122,  0.0620008 ,  0.07276516,
        0.06415575]), 0.25: array([15.06591952, 14.74258081, 14.99230181,  0.08355443,  0.08561355,
        0.08396665])}
Ridge_lcdf_kernel Target RMSE
{1.0: array([1.51905341, 2.06275147, 1.63040385, 0.44271821, 0.62213614,
       0.47863633]), 0.75: array([2.19245196, 2.57954095, 2.27138249, 0.63897583, 0.77800242,
       0.66680791]), 0.5: array([2.50723751, 2.78201543, 2.5629707 , 0.73071803, 0.83906974,
       0.75240923]), 0.25: array([3.8251488 , 3.57944404, 3.77342258, 1.11481469, 1.07957818,
       1.1077606 ])}


In [107]:
print ("Ridge_lcdf_soap DOS RMSE")
print (Ridge_lcdf_soap.DOSrmse)
print ("Ridge_lcdf_soap Target RMSE")
print (Ridge_lcdf_soap.targetrmse)

Ridge_lcdf_soap DOS RMSE
{1.0: array([12.5041179 , 13.05041966, 12.61212243,  0.06934688,  0.07578678,
        0.0706361 ]), 0.75: array([13.0034001 , 13.53473352, 13.10808214,  0.07211586,  0.0785993 ,
        0.07341379]), 0.5: array([13.97558488, 14.06515944, 13.9881487 ,  0.07750752,  0.08167961,
        0.07834274]), 0.25: array([16.73216006, 15.43007066, 16.45465778,  0.09279527,  0.08960596,
        0.0921568 ])}
Ridge_lcdf_soap Target RMSE
{1.0: array([3.14754043, 3.16554191, 3.15026236, 0.91733014, 0.95474324,
       0.92481996]), 0.75: array([3.31747847, 3.35290847, 3.32373203, 0.96685748, 1.01125393,
       0.97574531]), 0.5: array([3.67800992, 3.57160966, 3.65498061, 1.07193202, 1.07721529,
       1.07298969]), 0.25: array([4.50041176, 3.95253412, 4.3863266 , 1.31161567, 1.19210401,
       1.28769034])}


In [108]:
print ("GPR_lcdf DOS RMSE")
print (GPR_lcdf.DOSrmse)
print ("GPR_lcdf Target RMSE")
print (GPR_lcdf.targetrmse)

GPR_lcdf DOS RMSE
{1.0: array([20.90363469, 20.46987494, 20.80457749,  0.11592995,  0.11887326,
        0.11651918]), 0.75: array([22.40731733, 22.02863113, 22.25118271,  0.12361527,  0.12792531,
        0.12462111]), 0.5: array([22.55960419, 22.07145294, 22.64809633,  0.12511565,  0.12817399,
        0.12684409]), 0.25: array([23.69955108, 22.92802315, 23.60884309,  0.125426  ,  0.13314829,
        0.13222489])}
GPR_lcdf Target RMSE
{1.0: array([5.63197171, 5.46999487, 5.59690318, 1.64140145, 1.64977775,
       1.64307832]), 0.75: array([6.08147253, 5.83454171, 5.94154172, 1.75597532, 1.75972689,
       1.74425358]), 0.5: array([6.05751721, 6.14648197, 6.31087866, 1.79060255, 1.85380963,
       1.85267952]), 0.25: array([6.62111657, 6.19329568, 6.35071534, 1.75255165, 1.86792888,
       1.86437434])}


## Torch Model

In [283]:
import dostools.src.models.models as models
import dostools.src.models.training as training
import dostools.src.models.architectures as architecture
import dostools.src.loss.loss as loss
importlib.reload(models)
importlib.reload(training)
importlib.reload(architecture)
importlib.reload(loss)

TLReg_pw_soap = models.TRegression(Silicon.Features['structure_avedescriptors'].double(), "avedesc", y_pw.double(), "pw", "pw", xdos, "LBFGS", "cpu")
TLReg_pw_kernel = models.TRegression(Silicon.Features['structure_avekerneldescriptors'].double(), "avekerneldesc", y_pw.double(), "pw", "pw", xdos, "LBFGS", "cpu")
TLReg_lcdf_soap = models.TRegression(Silicon.Features['structure_avedescriptors'].double(), "avedesc", y_lcdf.double(), "lcdf", "pw", xdos, "LBFGS", "cpu")
TLReg_lcdf_kernel = models.TRegression(Silicon.Features['structure_avekerneldescriptors'].double(), "avekerneldesc", y_lcdf.double(), "lcdf", "pw", xdos, "LBFGS", "cpu")


In [284]:
torch.manual_seed(0)
TLReg_pw_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index, 1e-11, 1, 20000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_pw_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index, 2.5119e-08, 1, 20000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_lcdf_soap.obtain_weights([1.0,0.75,0.5,0.25], train_index, 1e-10, 1, 20000, loss.t_get_mse)
torch.manual_seed(0)
TLReg_lcdf_kernel.obtain_weights([1.0,0.75,0.5,0.25], train_index, 3.1623e-10, 1, 20000, loss.t_get_mse)

Currently training torch linear model with features :avedesc, target: pw, opt: LBFGS, lr:1


Epoch: 63:   0%|▎                                                                                                     | 63/20000 [01:18<6:53:11,  1.24s/it, lowest_loss=0.944, pred_loss=0.936, trigger=30]


Implemented early stopping with lowest_loss: 0.9442992561216077
Currently training torch linear model with features :avedesc, target: pw, opt: LBFGS, lr:1


Epoch: 56:   0%|▎                                                                                                       | 56/20000 [01:04<6:22:51,  1.15s/it, lowest_loss=1.01, pred_loss=1.01, trigger=30]


Implemented early stopping with lowest_loss: 1.0097232858565601
Currently training torch linear model with features :avedesc, target: pw, opt: LBFGS, lr:1


...

Implemented early stopping with lowest_loss: 9.00176011701582
Currently training torch linear model with features :avedesc, target: pw, opt: LBFGS, lr:1


Epoch: 54:   0%|▎                                                                                                      | 54/20000 [03:16<20:08:37,  3.64s/it, lowest_loss=10.3, pred_loss=10.3, trigger=30]


Implemented early stopping with lowest_loss: 10.348100292758737
Currently training torch linear model with features :avedesc, target: pw, opt: LBFGS, lr:1


Epoch: 62:   0%|▎                                                                                                      | 62/20000 [03:47<20:20:03,  3.67s/it, lowest_loss=10.8, pred_loss=10.8, trigger=30]

Implemented early stopping with lowest_loss: 10.811720761936554


In [70]:
Ridge_pw = models.RidgeRegression(Silicon.Features['structure_avedescriptors'], "avedesc", y_pw, "pw", xdos)
Ridge_pw.obtain_weights([1.0,0.5], train_index)
Ridge_pw.get_DosRMSE(evaluator, train_index, test_index)
Ridge_pw.get_targetRMSE(evaluator, train_index, test_index)


The performance at reg value 9.999999999999996e-11 is 0.0470273564726883
The performance at reg value 9.999999999999982e-09 is 0.03135393777242554
The performance at reg value 1.0000000000000004e-06 is 0.020054425087597604
The performance at reg value 0.00010000000000000009 is 0.016362072888277648
The performance at reg value 0.010000000000000004 is 0.02395674318210591
The performance at reg value 1.0 is 0.04281414485376572
The performance at reg value 100.00000000000004 is 0.08142463483743637
The log_grid error is:
[0.04702736 0.03135394 0.02005443 0.01636207 0.02395674 0.04281414
 0.08142463]
Init value is: 0.00010000000000000009
The performance at reg value 0.00010000000000000009 is 0.016362072888277648
The performance at reg value 6.309573444801939e-05 is 0.016439094014605245
The performance at reg value 0.00015848931924611147 is 0.016417027215670447
The performance at reg value 0.00012589254117941672 is 0.01637212888920042
Optimal regularization value for Ridge, Feature:avedesc, t

In [287]:
TLReg_pw_soap.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_pw_kernel.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_soap.get_DosRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_kernel.get_DosRMSE(evaluator, train_index, test_index)
TLReg_pw_soap.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_pw_kernel.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_soap.get_targetRMSE(evaluator, train_index, test_index) 
TLReg_lcdf_kernel.get_targetRMSE(evaluator, train_index, test_index)

In [295]:
print ("soap pw")
print (TLReg_pw_soap.models[1.0].reg)
print (TLReg_pw_soap.DOSrmse)
print (TLReg_pw_soap.targetrmse)

soap
tensor(1.0000e-11)
{1.0: array([11.10363366, 11.84994029, 11.25375874,  0.0615799 ,  0.06881532,
        0.06302838]), 0.75: array([11.39326201, 12.08773556, 11.53250281,  0.06318616,  0.07019625,
        0.06458952]), 0.5: array([11.08235802, 11.82524186, 11.23178179,  0.06146191,  0.06867189,
        0.06290529]), 0.25: array([11.3848421 , 12.0876758 , 11.5258219 ,  0.06313946,  0.07019591,
        0.06455211])}
{1.0: array([11.10363366, 11.84994029, 11.25375874,  0.0615799 ,  0.06881532,
        0.06302838]), 0.75: array([11.39326201, 12.08773556, 11.53250281,  0.06318616,  0.07019625,
        0.06458952]), 0.5: array([11.08235802, 11.82524186, 11.23178179,  0.06146191,  0.06867189,
        0.06290529]), 0.25: array([11.3848421 , 12.0876758 , 11.5258219 ,  0.06313946,  0.07019591,
        0.06455211])}


In [296]:
print ("soap lcdf")
print (TLReg_lcdf_soap.models[1.0].reg)
print (TLReg_lcdf_soap.DOSrmse)
print (TLReg_lcdf_soap.targetrmse) <-???

soap lcdf
tensor(1.0000e-10)
{1.0: array([5250.50813994, 5145.91094413, 5226.53088909,   29.11891488,
         29.88348498,   29.27197607]), 0.75: array([5251.75167346, 5146.74725943, 5227.68935715,   29.12581142,
         29.88834166,   29.27846425]), 0.5: array([5250.56392374, 5145.85978877, 5226.56445077,   29.11922425,
         29.88318791,   29.27216404]), 0.25: array([5250.45447671, 5145.62601985, 5226.42924465,   29.11861727,
         29.88183036,   29.2714068 ])}
{1.0: array([5250.50813994, 5145.91094413, 5226.53088909,   29.11891488,
         29.88348498,   29.27197607]), 0.75: array([5251.75167346, 5146.74725943, 5227.68935715,   29.12581142,
         29.88834166,   29.27846425]), 0.5: array([5250.56392374, 5145.85978877, 5226.56445077,   29.11922425,
         29.88318791,   29.27216404]), 0.25: array([5250.45447671, 5145.62601985, 5226.42924465,   29.11861727,
         29.88183036,   29.2714068 ])}


In [294]:
print ("kernel pw ")
print (TLReg_pw_kernel.models[1.0].reg)
print (TLReg_pw_kernel.DOSrmse)
print (TLReg_pw_kernel.targetrmse)

kernel
tensor(2.5119e-08)
{1.0: array([18.45811321, 18.01306498, 18.35776791,  0.10236728,  0.104606  ,
        0.10281545]), 0.75: array([17.1342939 , 16.9566007 , 17.09001638,  0.09502548,  0.09847087,
        0.09571522]), 0.5: array([17.42045947, 17.147869  , 17.35636004,  0.09661253,  0.09958161,
        0.09720692]), 0.25: array([17.17396184, 16.98331934, 17.12697923,  0.09524547,  0.09862603,
        0.09592224])}
{1.0: array([18.45811321, 18.01306498, 18.35776791,  0.10236728,  0.104606  ,
        0.10281545]), 0.75: array([17.1342939 , 16.9566007 , 17.09001638,  0.09502548,  0.09847087,
        0.09571522]), 0.5: array([17.42045947, 17.147869  , 17.35636004,  0.09661253,  0.09958161,
        0.09720692]), 0.25: array([17.17396184, 16.98331934, 17.12697923,  0.09524547,  0.09862603,
        0.09592224])}


In [297]:
print ("kernel lcdf ")
print (TLReg_lcdf_kernel.models[1.0].reg)
print (TLReg_lcdf_kernel.DOSrmse)
print (TLReg_lcdf_kernel.targetrmse)

kernel lcdf 
tensor(3.1623e-10)
{1.0: array([5245.2327782 , 5141.31965511, 5221.39980085,   29.08965813,
         29.85682231,   29.24323864]), 0.75: array([5244.5669767 , 5138.31529928, 5220.24885762,   29.08596564,
         29.83937533,   29.23679261]), 0.5: array([5243.3780059 , 5140.54490559, 5219.77001617,   29.0793717 ,
         29.85232317,   29.23411078]), 0.25: array([5245.13358744, 5140.27060078, 5221.1034835 ,   29.08910802,
         29.85073022,   29.24157907])}
{1.0: array([5245.2327782 , 5141.31965511, 5221.39980085,   29.08965813,
         29.85682231,   29.24323864]), 0.75: array([5244.5669767 , 5138.31529928, 5220.24885762,   29.08596564,
         29.83937533,   29.23679261]), 0.5: array([5243.3780059 , 5140.54490559, 5219.77001617,   29.0793717 ,
         29.85232317,   29.23411078]), 0.25: array([5245.13358744, 5140.27060078, 5221.1034835 ,   29.08910802,
         29.85073022,   29.24157907])}


Multiplying loss by col A : Train error col B
1e0 = 16.188
1e3 = 11.73
1e5 = 11.79
1e7 = 11.41
1e10 = 11.66
1e20 = 11.27
1e90 = 11.29

{1.0: array([11.41705718, 12.11622485, 11.5572621 ,  0.06331812,  0.0703617 ,
         0.06472819])}

In [28]:
with torch.no_grad():
    GPR_pred = Silicon.Features['structure_avekerneldescriptors'] @ GPR_pw.weights[1.0]
    Ridge_pred = Ridge_pw.models[1.0].predict(Silicon.Features['structure_avekerneldescriptors'])
    torch_pred = TLReg.models[1.0](Silicon.Features['structure_avedescriptors'].double())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1039x448 and 1000x778)

In [ ]:
plt.plot(xdos, GPR_pred[369], color = 'r', label = "GPR_pred")
plt.plot(xdos, Ridge_pred[369], color = 'g', label = "Ridge_pred")
plt.plot(xdos, torch_pred[369], color = 'c', label = "Torch_pred")
plt.plot(xdos, y_pw[369], color = 'b', label = 'true')
plt.legend()

